In [3]:
import pandas as pd
import glob
import sys
sys.path.insert(1, '..')
from src import readfiles
from pathlib import Path
import numpy as np

# **Warning:**
# Files named 001 and 002 do not always correspond to the same crystal!
# So this information must be extracted from the metadata.
# So get all the data into one big list!

def read_scans_and_split_by_xtl(allfolders):

    data_list = []
    metadata_list = []

    for i in range(len(allfolders)):
        _, data, metadata = readfiles.import_jeol_wdscans(allfolders[i],
                                                             'data001_mm.csv', 'data001.cnd',
                                                             comment_line_num=80,
                                                             return_metadata=True)
        data_list.append(data)
        metadata_list.append(metadata)
    
        _, data, metadata = readfiles.import_jeol_wdscans(allfolders[i],
                                                             'data002_mm.csv', 'data002.cnd',
                                                             comment_line_num=80,
                                                             return_metadata=True)
        data_list.append(data)
        metadata_list.append(metadata)
    
    # And then split based on crystal, and save to 'cleaned_data' folder.
    
    xtls = list(set([m.crystal for m in metadata_list])) # Find the unique crystals
    data_by_xtl = {xtls[0]: [], xtls[1]: []}
    
    for i, m in enumerate(metadata_list):
    
        data_by_xtl[m.crystal].append(data_list[i])

    return data_by_xtl

def average_spectra(data):

    """ data = a list of pandas dataframes """

    # Rehape data into np arrays
    cpsdata = np.empty(shape=(len(data[0].cps), len(data)))
    cps_per_nA_data = np.empty(shape=(len(data[0].cps), len(data)))
    Ldata = np.empty(shape=(len(data[0].L), len(data)))

    for i in range(len(data)):
        cpsdata[:, i] = data[i].cps
        cps_per_nA_data[:,i] = data[i].cps_per_nA
        Ldata[:, i] = data[i].L

    # Check L-axes are the same:
    L_axis_check = np.array([Ldata[:,n] == Ldata[:,0]
                             for n in range(len(data))])


    if L_axis_check.all().all() == False:
        print(Ldata[1:10, :])
        raise Exception('L values are not the same between datasets!')

    # Average the data
    mean_df = pd.DataFrame({'L': Ldata[:,0],
                            'cps': cpsdata.mean(axis=1),
                            'cps_per_nA': cps_per_nA_data.mean(axis=1),
                            'cps_per_nA_stdev': cps_per_nA_data.std(axis=1)})

    return mean_df

# Compile the ANU data and save

In [5]:
budd_folders = sorted(glob.glob('../data/raw/peak_shapes/raw_data_ANU/Buddingtonite/*'))
budd_data = read_scans_and_split_by_xtl(budd_folders)
budd_avg = {'LDE1': average_spectra(budd_data['LDE1']),
             'LDE1H': average_spectra(budd_data['LDE1H'])}

In [9]:
glass_folders = sorted(glob.glob('../data/raw/peak_shapes/raw_data_ANU/GlassA870/*'))
glass_data = read_scans_and_split_by_xtl(glass_folders)
glass_avg = {'LDE1': average_spectra(glass_data['LDE1']),
             'LDE1H': average_spectra(glass_data['LDE1H'])}

In [14]:
nitride_folders = sorted(glob.glob('../data/raw/peak_shapes/raw_data_ANU/*'))
nitride_folders = [n for n in nitride_folders if 'Buddingtonite' not in n if 'Glass' not in n]
nitride_names = [Path(n).stem for n in nitride_folders]
nitride_data = read_scans_and_split_by_xtl(nitride_folders)

In [16]:
for xtl in ['LDE1', 'LDE1H']:

    # Check if folder exists, if not, create it
    output_folder = Path(f'../data/interim/peak_shapes/cleaned_data_ANU/{xtl}/')
    
    if not output_folder.exists():
        output_folder.mkdir(parents=True, exist_ok=True)
    
    for i in range(len(nitride_data[xtl])):
        nitride_data[xtl][i].to_csv(output_folder / Path(nitride_names[i] + '.csv'))
    
    budd_avg[xtl].to_csv(output_folder / Path('Buddingtonite.csv'))
    glass_avg[xtl].to_csv(output_folder / Path('GlassA870.csv'))


# Compile the StA data and save

In [26]:
allfolders = sorted(glob.glob('../data/raw/peak_shapes/raw_data_StA/*'))

names = [Path(n).stem for n in allfolders]
data = read_scans_and_split_by_xtl(allfolders)

for xtl in data.keys():

    # Check if folder exists, if not, create it
    output_folder = Path(f'../data/interim/peak_shapes/cleaned_data_StA/{xtl}/')
    
    if not output_folder.exists():
        output_folder.mkdir(parents=True, exist_ok=True)
    
    for i in range(len(data[xtl])):
        data[xtl][i].to_csv(output_folder / Path(names[i] + '.csv'))


# 